## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [5]:
!curl -L "https://app.roboflow.com/ds/oVBg5uGOAb?key=EmAK5tCOK5" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/18/118_png.rf.ca711bcf43fc1a937ca9844ce07535fa.jpg  
 extracting: train/18/118_png.rf.fe1ec052fb4a80b328fc8d78dabce5f4.jpg  
 extracting: train/18/1190_png.rf.a0b7aa8699ed0b06d2a2e4e29a01c38d.jpg  
 extracting: train/18/1190_png.rf.d4eba5804dcc036e93395ed923203a85.jpg  
 extracting: train/18/1190_png.rf.e932be15f947cfbae994cf0e670e77a9.jpg  
 extracting: train/18/1191_png.rf.1df190f90725c9b964f10ec76cc52119.jpg  
 extracting: train/18/1191_png.rf.1f0c3b3e11c5eb1c05f94d29fdf17019.jpg  
 extracting: train/18/1191_png.rf.8f0b9f6493899e8c5c601fa0d47123ae.jpg  
 extracting: train/18/1192_png.rf.12a014b8af4b91417e3fbc2bde1a795c.jpg  
 extracting: train/18/1192_png.rf.1a1597ec1435ed8f27b89be6db675c7f.jpg  
 extracting: train/18/1192_png.rf.ab5cef980433787d125e8c6c3ec956e6.jpg  
 extracting: train/18/1193_png.rf.242eac2539b6d25ae7b00d9e0c0a6e47.jpg  
 extracting: train/18/1193_png.rf.45cf9211c151ffb9698cec55adde7772.jpg  
 extracting: trai

In [6]:
all_img_list = glob.glob('/content/train/*/*')

In [7]:
all_img_list2 = glob.glob('/content/valid/*/*')

In [8]:
len(all_img_list)

7182

In [9]:
all_img_list = all_img_list + all_img_list2

In [10]:
len(all_img_list)

8217

In [11]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[3])

In [12]:
df

,img_path,label
0,/content/train/10/496_png.rf.d22c97db805cf2a43...,10
1,/content/train/10/338_png.rf.4afce372e99de2bfc...,10
2,/content/train/10/414_png.rf.d1f33bcedb5911743...,10
3,/content/train/10/138_png.rf.4c4a661cfdc8bb0af...,10
4,/content/train/10/501_png.rf.057b50a208c817b9d...,10
...,...,...
8212,/content/valid/13/14_png.rf.f3731a7a62142fd562...,13
8213,/content/valid/8/2_png.rf.40ca0bf4fa728eb093ac...,8
8214,/content/valid/4/1_png.rf.4b7899f321a5a39cc2eb...,4
8215,/content/valid/4/4_png.rf.f88ad4c340b3865c8329...,4


In [13]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [14]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

In [15]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [16]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [17]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [18]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [19]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='wigehted')
    
    return _val_loss, _val_score

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.98287] Val Loss : [0.59732] Val Weighted F1 Score : [0.79171]


  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

...

Epoch [3], Train Loss : [0.13871] Val Loss : [0.46588] Val Weighted F1 Score : [0.87174]


  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [4], Train Loss : [0.10876] Val Loss : [0.60636] Val Weighted F1 Score : [0.84847]


  0%|          | 0/206 [00:00<?, ?it/s]

## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)